In [ ]:
import json

import pandas as pd
import requests

import my_config

class RestAPI:
    def __init__(self, base_url, endpoints):
        self.base_url = base_url
        self.endpoints = endpoints
        self.data = {}

    def fetch_data(self):
        for key, endpoint in self.endpoints.items():
            url = self.base_url + endpoint
            response = requests.get(url)
            if response.status_code == 200: self.data[key] = response.json()

US Bureau of Economic Analysis API

In [ ]:
all_bea_tables = {
    "T10503": "Real Gross Domestic Product, Expanded Detail, Quantity Indexes (A) (Q)",
    "T10105": "Gross Domestic Product (A) (Q)",
    "T10505": "Gross Domestic Product, Expanded Detail (A) (Q)",
    "T80103": "Real Gross Domestic Product, Quantity Indexes, Not Seasonally Adjusted (Q)",
    "T80104": "Price Indexes for Gross Domestic Product, Not Seasonally Adjusted (Q)",
    "T80105": "Gross Domestic Product, Not Seasonally Adjusted (Q)",
    "T80106": "Real Gross Domestic Product, Chained Dollars, Not Seasonally Adjusted (Q)",
    "T80200": "Gross Domestic Income by Type of Income, Not Seasonally Adjusted (Q)",
    "T11000": "Gross Domestic Income by Type of Income (A) (Q)",
    "T11000": "Gross Domestic Income by Type of Income (A) (Q)",

    "T20301": "Percent Change From Preceding Period in Real Personal Consumption Expenditures by Major Type of Product (A) (Q)",

    "T20100": "Personal Income and Its Disposition (A) (Q)",
    "T20600": "Personal Income and Its Disposition, Monthly (M)",
    "T20200A": "Wages and Salaries by Industry (A) (Q)",
    "T20200B": "Wages and Salaries by Industry (A) (Q)",
    "T20700A": "Wages and Salaries by Industry, Monthly (M)",
    "T20700B": "Wages and Salaries by Industry, Monthly (M)",

    "T40201": "Percent Change From Preceding Period in Real Exports and in Real Imports of Goods and Services by Type of Product (A) (Q)",
    "T40203": "Real Exports and Imports of Goods and Services by Type of Product, Quantity Indexes (A) (Q)",
    "T40204": "Price Indexes for Exports and Imports of Goods and Services by Type of Product (A) (Q)",
    "T40205": "Exports and Imports of Goods and Services by Type of Product (A) (Q)",

    "T50100": "Saving and Investment by Sector (A) (Q)",
    "T50203": "Real Gross and Net Domestic Investment by Major Type, Quantity Indexes (A)",
    "T50205": "Gross and Net Domestic Investment by Major Type (A)",
    "T50301": "Percent Change From Preceding Period in Real Private Fixed Investment by Type (A) (Q)",

    "T50705A": "Change in Private Inventories by Industry (A) (Q)",
    "T50705B": "Change in Private Inventories by Industry (A) (Q)",

    "T61600A": "Corporate Profits by Industry (A)",
    "T61600B": "Corporate Profits by Industry (A) (Q)",
    "T61600C": "Corporate Profits by Industry (A) (Q)",
    "T61600D": "Corporate Profits by Industry (A) (Q)",

    "T70201A": "Percent Change From Preceding Period in Real Auto Output (A) (Q)",
    "T70201B": "Percent Change From Preceding Period in Real Motor Vehicle Output (A) (Q)",
    "T70203A": "Real Auto Output, Quantity Indexes (A) (Q)",
    "T70203B": "Real Motor Vehicle Output, Quantity Indexes (A) (Q)",

    "T80300": "Federal Government Current Receipts and Expenditures, Not Seasonally Adjusted (Q)"
}

gdp_percent_changes = {
    "T10101" : "Percent Change From Preceding Period in Real Gross Domestic Product (A) (Q)",
    "T10107": "Percent Change From Preceding Period in Prices for Gross Domestic Product (A) (Q)",
    "T10108": "Contributions to Percent Change in the Gross Domestic Product Price Index (A) (Q)",
    "T10501": "Percent Change From Preceding Period in Real Gross Domestic Product, Expanded Detail (A) (Q)",
    "T80111": "Real Gross Domestic Product: Percent Change From Quarter One Year Ago, Not Seasonally Adjusted (Q)"
}

In [ ]:
start_year = 2020
end_year = 2023
bea_years = range(start_year, end_year + 1)
bea_years = ','.join(str(year) for year in bea_years)

bea_frequency = 'Q'
bea_dataset = 'NIPA'
selected_tables = ['T10105', 'T20200B']
# selected_tables = ['T10107', 'T80111', 'T10105', 'T20200B', 'T20100', 'T61600A']

if 'T10101' not in selected_tables: 
    selected_tables += ['T10101']
    T10101_added = True

bea_base_url = f"https://apps.bea.gov/api/data/"

# The multiselect components' value: datasets, is a list of table names.
# We can use dictionary comprehension to create the endpoints from this, with the table names are used as the keys.
bea_endpoints = {f'{table}': f"?&UserID={my_config.BEA_KEY} \
                                  &method=GetData \
                                  &DataSetName={bea_dataset} \
                                  &TableName={table} \
                                  &Frequency={bea_frequency} \
                                  &Year={bea_years}" for table in selected_tables}

# In general, each table contains multiple economic metrics, for example, gross domestic product and gross national product.
# We can use a dictionary to filter on the metric we're interested in per table rather than doing several if statements.
filter_metrics = {
    "T10101": "Gross domestic product",
    'T10105': "Gross domestic product",
    'T10107': "Gross domestic product",
    'T20100': "test",
    'T20200B': "Wages and salaries",
    'T61600A': "test"
}

bea_api = RestAPI(bea_base_url, bea_endpoints)
bea_api.fetch_data()

# print(json.dumps(bea_api.data['T20200B']['BEAAPI']['Results']['Data'], indent=4))

# Creating a date series to index the user selected DataFrames.
periods = [item['TimePeriod'] for item in bea_api.data['T10101']['BEAAPI']['Results']['Data']]
unique_periods = list(set(periods))
date_sr = pd.Series(unique_periods, name='date').sort_values().reset_index(drop=True)

if T10101_added: selected_tables.remove('T10101')

# Replacing the JSON data within the bea_api.data dictionary with formatted DataFrames.
bea_dfs = [date_sr]
for table in selected_tables:
    bea_api.data[table] = (
        pd.DataFrame(bea_api.data[table]['BEAAPI']['Results']['Data'], columns=['DataValue', 'METRIC_NAME', 'LineDescription']) \
        .rename(columns={'DataValue': table + '_value', 'METRIC_NAME': table + '_metric'}))
    bea_api.data[table] = (
        bea_api.data[table].loc[bea_api.data[table]['LineDescription'] == filter_metrics[table]]
        .drop(columns=['LineDescription']))

    # The tables contain multiple values for a given economic measure using differnt methodologies. These are vertically stacked.
    # To avoid needing to filter on the method column, the table is pivotted in a way that doesn't produce NaN values:
    sliced_dfs = []
    unique_metrics = bea_api.data[table][table + '_metric'].unique()
    for metric in unique_metrics:
        sliced_df = (
            bea_api.data[table].loc[bea_api.data[table][table + '_metric'] == metric]
            .drop(columns=[table + '_metric'])
            .reset_index(drop=True)
            .rename(columns={f'{table}_value': f'{table} - ' + metric})
        )
        sliced_dfs.append(sliced_df)
    merged_df = pd.concat(sliced_dfs, axis=1)
    bea_dfs.append(merged_df)
bea_df = pd.concat(bea_dfs, axis=1)

print(bea_df)

In [ ]:
# Save json output to a .json file.
json_api_2 = json.dumps(bea_api.data['annual_gdp']['BEAAPI']['Results']['Data'], indent=4)
func = open("dict_2.json","w")
func.write(json_api_2)
func.close()

US Treasury Department API

In [ ]:
'''
Treasury Endpoint Descriptions:
"interest_rates" - Average interest rates for marketable and non-marketable securities.
"cash_balance" - This table represents the Treasury General Account balance. All figures are rounded to the nearest million.
"transactions" - This table represents deposits and withdrawals from the Treasury General Account. 
"us_debt" - Outstanding U.S. debt on a daily basis. Measured to the penny.
All figures are rounded to the nearest million.
'''

treasury_base_url = "https://api.fiscaldata.treasury.gov/services/api/fiscal_service/"
treasury_endpoints = {
    'interest_rates': 'v2/accounting/od/avg_interest_rates',
    'cash_balance': 'v1/accounting/dts/dts_table_1',
    'transactions': 'v1/accounting/dts/dts_table_2',
    'us_debt': 'v2/accounting/od/debt_to_penny',
    'balance_sheet': 'v2/accounting/od/balance_sheets'
}

treasury_api = RestAPI(treasury_base_url, treasury_endpoints)
treasury_api.fetch_data()

print(json.dumps(treasury_api.data, indent=4))

US Bureau of Labor Statistics API - really slow

In [ ]:
labor_base_url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'

labor_endpoints = {
    'unemployment_rate': 'LNS14000000',
    # 'cpi': 'CUUR0000SA0'
}

labor_dates = {
    'startyear': '2005',
    'endyear': '2020'
}

labor_api = RestAPI(labor_base_url, labor_endpoints)
labor_api.fetch_data()

print(json.dumps(labor_api.data['unemployment_rate'], indent=4))